## TASK 3

In [1]:
import pandas_gbq
import os
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import storage
import numpy as np
import sqlite3





In [2]:
# Read CSVs in chunks and upload to BigQuery
credentials_path = r"C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project-bt-b14310631abc.json"
credentials = service_account.Credentials.from_service_account_file(credentials_path)
gbq_proj_id = "wedge-project-bt"
dataset_id = "wedge_transactions"

In [3]:
#Query 1

query = """SELECT
  EXTRACT(DATE
  FROM
    datetime) AS date,
    EXTRACT(HOUR FROM datetime) AS hour,
 ROUND(SUM(total),2) AS spend,
  COUNT(DISTINCT CONCAT(EXTRACT(DATE
        FROM
          datetime), 
          register_no, emp_no, trans_no)) AS trans,
  SUM(CASE
      WHEN trans_status IN ('V', 'R') THEN -1
    ELSE
    1
  END
    ) AS items
FROM
  `wedge_transactions.transArchive*`
WHERE
  department NOT IN (0,
    15)
  AND (trans_status IS  NULL
  OR trans_status IN (' ','V','R'))
GROUP BY
  date, hour
  ORDER BY 
  date, hour;
  """


In [4]:
# Path to service account JSON key file
conn = sqlite3.connect('wedge-reporting.db')


In [7]:
holder = pandas_gbq.read_gbq(query, project_id=gbq_proj_id, credentials=credentials)    

GenericGBQException: Reason: 404 Not found: Dataset wedge-project-bt:wedge_transactions was not found in location US

Location: US
Job ID: 456b5ba4-8a91-401c-b133-40be6839eaf3


In [ ]:
holder.head()

In [ ]:
holder.to_sql('date-hour', conn, if_exists='replace', index=False)

In [ ]:
#Query 2 

query2 = """SELECT DISTINCT card_no,
   EXTRACT(YEAR FROM datetime) AS year,
    EXTRACT(MONTH FROM datetime) AS month,
  SUM(total) AS spend,
  COUNT(DISTINCT CONCAT(EXTRACT(DATE
        FROM
          datetime), 
          register_no, emp_no, trans_no)) AS trans,
  SUM(CASE
      WHEN trans_status IN ('V', 'R') THEN -1
    ELSE
    1
  END
    ) AS items
FROM
  `wedge_transactions.transArchive*`
WHERE
  department NOT IN (0,
    15)
  AND  card_no != 3
  AND (trans_status IS  NULL
  OR trans_status IN (' ','V','R'))
GROUP BY
  card_no, year, month
  ORDER BY 
 card_no, year, month;
  """


In [ ]:
holder = pandas_gbq.read_gbq(query2, project_id=gbq_proj_id, credentials=credentials)    

In [ ]:
holder.head()

In [ ]:
holder.to_sql('owner-sales', conn, if_exists='replace', index=False) 

In [ ]:
#Query 3 

query3 = """SELECT upc,
  description,
  w.department,
  dept_name,
   EXTRACT(YEAR FROM datetime) AS year,
    EXTRACT(MONTH FROM datetime) AS month,
  ROUND(SUM(total),2) AS sales,
  COUNT(DISTINCT CONCAT(EXTRACT(DATE
        FROM
          datetime), 
          register_no, emp_no, trans_no)) AS trans,
  SUM(CASE
      WHEN trans_status IN ('V', 'R') THEN -1
    ELSE
    1
  END
    ) AS items
FROM
  `wedge_transactions.transArchive*` AS w
JOIN `wedge_transactions.department_lookup` AS d
ON w.department = d.department
WHERE
  w.department NOT IN (0,
    15)
  AND  card_no != 3
  AND (trans_status IS  NULL
  OR trans_status IN (' ','V','R'))
GROUP BY
  upc,description,w.department, dept_name, year, month 
ORDER BY 
 sales DESC, description, year, month;
  """


In [ ]:
holder = pandas_gbq.read_gbq(query3, project_id=gbq_proj_id, credentials=credentials)    

In [ ]:
holder.head()

In [ ]:
holder.to_sql('product-sales', conn, if_exists='replace', index=False) 